In [91]:
import pickle
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network
import matplotlib.pyplot as plt

In [92]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pickle.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [93]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 100)    4400        ['input_11[0][0]']               
                                                                                                  
 embedding_11 (Embedding)       (None, None, 100)    1900        ['input_12[0][0]']               
                                                                                           

In [94]:
pieces_train = None
with open('../data/Oskar Kolberg\'s Dataset/train/pieces_train.pkl', 'rb') as handle:
    pieces_train = pickle.load(handle)

pieces_test = None
with open('../data/Oskar Kolberg\'s Dataset/train/pieces_test.pkl', 'rb') as handle:
    pieces_test = pickle.load(handle)

In [ ]:
def get_boundaries(piece):
    notes_input = [ notes_to_int[note] for note in piece[0] ]
    durations_input = [ durations_to_int[duration] for duration in piece[1] ]
    expected = phrases_input = piece[2]
    
    max_extra_notes = len(notes_input) - 1
    seq_len = 5
    phrases_output = []
    phrases_output.append(0) # after the first note (that is already in the initial sequence)

    notes_input_sequence = [notes_input[0]] # the first note element of the sequence
    durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

    if seq_len is not None:
        notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
        durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

    for note_index in range(max_extra_notes):
        prediction_input = [
            np.array([notes_input_sequence])
            , np.array([durations_input_sequence])
        ]

        phrases_prediction = model.predict(prediction_input, verbose=0)
        phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

        notes_input_sequence.append(notes_input[note_index])
        durations_input_sequence.append(durations_input[note_index])
        
        if len(notes_input_sequence) > seq_len:
            notes_input_sequence = notes_input_sequence[1:]
            durations_input_sequence = durations_input_sequence[1:]
    actual = phrases_output
    
    return (actual, expected)

### Calculate F1 score

In [5]:
def f1_score(expected, actual):
    """
    Calculates the F1 score given two arrays of 0s and 1s (expected and actual).
    """
    # Ensure that the inputs are numpy arrays.
    expected = np.array(expected)
    actual = np.array(actual)
    
    # Calculate the true positives, false positives, and false negatives.
    tp = np.sum((expected == 1) & (actual == 1))
    fp = np.sum((expected == 0) & (actual == 1))
    fn = np.sum((expected == 1) & (actual == 0))
    
    # Calculate the precision, recall, and F1 score.
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1

F1 for the train data

In [ ]:
actual = []
expected = []
for piece in pieces_train:
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
f1_score(expected,actual)

F1 for the test data

In [ ]:
actual = []
expected = []
for piece in pieces_test:
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
f1_score(expected,actual)